<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dtw-python

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LarsCV
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from math import sqrt
from dtw import *
%matplotlib inline

In [ ]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)

In [ ]:
CYAN = '#76ced6' ; LILA = '#777bd4'; VERDE='#17cb49'; NARA='#ff8000'; AZUL='#168fff'; OTROAZUL = "b-"; ROJO= "r-";
def print_serie2(serie_,prototipo_,title_,ytitle_,xtitle_,sizex_=7,sizey_=4,namefile_='figure.png'):
    fig, ax1 = plt.subplots(figsize=(sizex_,sizey_))
    plt.title(title_,fontsize='x-large',color=NARA)
    ax1.set_xlabel(xtitle_, color=NARA, fontsize='large')
    ax1.set_ylabel(ytitle_, color=NARA, fontsize='large')
    plt.tick_params(colors = NARA, which='both')
    ax1.spines['bottom'].set_color(NARA)
    ax1.spines['top'   ].set_color(NARA) 
    ax1.spines['right' ].set_color(NARA)
    ax1.spines['left'  ].set_color(NARA)
    if len(prototipo_) != 0: 
        plt.plot(prototipo_,alpha=0.6, linestyle='dashed', color='red', linewidth=3)
    for p in serie_:
        plt.plot(p,alpha=0.3, linewidth=2)    
    plt.savefig(namefile_, transparent=True)         
    plt.show()

In [ ]:
## Preparamos la información para graficar la importancia y las posiciones en Random Forest Regressor
def print_importances(model_,labels_):
    i=0
    labels = [str(x) for x in labels_]
    labels_importances = []
    for feature in model_.feature_importances_:
        labels_importances.append((feature,labels[i]))
        i=i+1   
    labels_importances.sort(key=lambda tup: tup[0], reverse=False)
    importances = []
    labels      = []
    for tup in labels_importances:
        importances.append(tup[0])
        labels.append(tup[1])
    fig, ax = plt.subplots()
    y_pos = np.arange(len(importances))
    ax.set_yticks(ticks=y_pos)
    ax.barh(labels, importances, align='center')
    ax.invert_yaxis()
    ax.set_xlabel('Importancia')
    ax.set_ylabel('Variable')
    ax.set_title( 'Importancia de las variables del bosque aleatorio')
    plt.show()

In [ ]:
## Encuentra los vecinos más cercanos
def get_neighbors(train, test_row, num_neighbors, typedist='euclidean'):
  
    distances = list()

    for train_row in train:
        if typedist == 'dtw':
            dist = dtw(test_row, train_row)
        else:
            dist = euclidean(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])

    neighbors = list()

    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

## Calcula distancia euclidiana
def euclidean(neig1, neig2):
	  distance = 0.0
	  for i in range(len(neig1)):
		    distance += (neig1[i] - neig2[i])**2
	  return sqrt(distance)

In [ ]:
#https://drive.google.com/drive/folders/1Gj3XK9kM-lE18uBMe3qrZOGEm8yAI8i9
#https://www.codegrepper.com/code-examples/python/how+to+read+csv+file+from+google+drive+on+google+colab+
path        = 'https://drive.google.com/uc?export=download&id=' 
URL_Demanda = 'https://drive.google.com/file/d/1xcpXDTE7H6EBMLOkic5lq-lzSwiLG2ZS/view?usp=sharing'
df_Demanda  = pd.read_csv(path + URL_Demanda .split('/')[-2], usecols=[0] ) #names=['CLVUNI','TYPE','NODE'], usecols=[1,2,3,4,5,6,], 1,2,3,4,5,6,7,8,9,10,11,12,13,15,
df_Demanda.dropna(inplace=True)
serie = df_Demanda.to_numpy()
serie = StandardScaler().fit_transform(serie)
serie = serie.ravel() ## Con esto quitamos el bracket o corchete en cada uno de loselementos del arreglo

In [ ]:
def analogo_knn(serie,vsel,vreg,k=10,tol=0.8,typedist='euclidian',typereg='OLS'):    
#    vsel       : tamanio de la ventana de selección
#    vreg       : tamanio de la ventana de regresión
#    k       : número de vecinos a buscar k
#    tol     : tolerancia de tamaño de ventanas para seleccion de vecinos
#    typedist: medida de distancia, 'dtw' o 'euclidian' o 'pearson'

    t_o = time.time()
    n = len(serie) ## longitud total de la serie

    ## PASO 1: Selección de las ventanas de mayor correlación.

    ## Calculamos la distancia entre todos los vecinos.
    v = vsel
    distances = []
    Y = serie[n-v:n]           ## últimos datos
    for i in range(n-2*v):
        if  typedist == 'dtw': ## dynamic time warping
            dist = dtw(Y, serie[i:i+v]).distance  
        elif typedist == 'pearson':
            dist = np.corrcoef(Y,serie[i:i+v])[1,0]
        else:
            dist = euclidean(Y,serie[i:i+v])
        if dist > 0:
            distances.append((i, dist))
        
    ## Calculamos el vecindario por distancia de menor a mayor y se guardan las posiciones.
    if typedist == 'pearson':
        ## En caso de pearson se ordena al revés, nos interesan los mayor correlación.
        distances.sort(key=lambda tup: tup[1], reverse=True)
    else:
        ## En caso de pearson se ordena al revés, nos interesan los de menor distancia.
        distances.sort(key=lambda tup: tup[1], reverse=False)

    neighbors  = []
    neighbors2 = []
    positions  = []

    ## Calculamos los k vecinos mas cercanos y guardamos las posiciones.
    i = 0
    for pos, dis in distances:
        if i==0:      
            positions.append(pos)   
            neighbors.append(serie[pos:pos+v])
            neighbors2.append(serie[pos+v:pos+2*v])
        else:
            bandera = True
            for p in positions:
                 ## si ya teniamos una posición en la lista que pase la tolerancia, ya no guardamos 
                if (abs(pos - p) < tol*v):
                    bandera = False
                    i = i - 1
                    break
            if bandera == True:
                ## Guarda nuevo vecino
                positions.append(pos)   
                neighbors.append(serie[pos:pos+v])
                neighbors2.append(serie[pos+v:pos+2*v])
                bandera = False
        i = i + 1
        if i == k:
            break
    #print('positions KNN:',positions) ## posición de los k vecinos mas cercanos

    neighbors  = np.array(neighbors)  
    neighbors2 = np.array(neighbors2)    
    vacia = []
    print_serie2(neighbors,Y, 'KNN - (X) - ' + typedist,'demanda','time',7,4,'fig_1')
    #print_serie2(neighbors2,vacia, 'KNN - X2 (prima)','demanda','time',7,4,'fig_2')

    t_sel = time.time() - t_o

    ## PASO 2: Regresión entre los vecinos mas cercanos 'X' y la última ventana 'Y'

    ## Definimos nuestros regresores   
    X   = (neighbors.T).tolist()
    X_2 = (neighbors2.T).tolist()
    Y   = (Y).tolist()
    prediction_Y2 = []

    ## Regresión -- OLS with Stepwise --
    if typereg == 'OLS':
        model   = sm.OLS(Y, X)
        results = model.fit()
        prediction_Y2 = results.predict(X_2)

        ## Ordenamos los valores 'pi' y se selecciona el más grande.
        i = 0
        pvalues = []
        for pi in results.pvalues:
            pvalues.append((i,pi))
            i = i + 1
        pvalues.sort(key=lambda tup: tup[1], reverse=True) ## Ordenamos por 'pi'
        (i, pi) = pvalues[0]  

        while pi > 0.001:
            X   = sm.add_constant(X)
            X_2 = sm.add_constant(X_2) 
            print('Retiramos regresor ---> X' + str(i))
            X   = np.delete(arr=X,   obj=i+0, axis=1)
            X_2 = np.delete(arr=X_2, obj=i+0, axis=1)   
            model   = sm.OLS(Y, X)
            results = model.fit()

            ## Ordenamos los valores 'pi' y se selecciona el más grande
            i = 0
            pvalues = []
            for pi in results.pvalues:
                pvalues.append((i,pi))
                i = i + 1
            pvalues.sort(key=lambda tup: tup[1], reverse=True) ## Ordenamos por 'pi'
            (i, pi) = pvalues[0]
            #prediction   = results.predict(X)   ## Ajuste
            prediction_Y2 = results.predict(X_2) ## Pronóstico
        if len(prediction_Y2) == 0:
            print('>>> Warning, no variable was significant in the regression.')
            model   = sm.OLS(Y, X)
            results = model.fit()
            prediction_Y2 = results.predict(X_2)

        print(results.summary())

    ## Voting regression 
    ## https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_regressor.html#sphx-glr-auto-examples-ensemble-plot-voting-regressor-py
    if typereg == 'Voting':
        reg1 = GradientBoostingRegressor(random_state=1)
        reg2 = RandomForestRegressor(random_state=1)
        reg3 = LinearRegression()
        reg1.fit(X, Y)
        reg2.fit(X, Y)
        reg3.fit(X, Y)
        ereg = VotingRegressor([("gb", reg1), ("rf", reg2), ("lr", reg3)])
        ereg.fit(X, Y)
        prediction_Y2 = ereg.predict(X_2)

    ## Regresión -- Random Forrest Regression 100 --
    if typereg == 'RF':
        model         = RandomForestRegressor(n_estimators=100,random_state=42,) ##default
        results       = model.fit(X, Y)
        prediction_Y2 = results.predict(X_2)
        print_importances(model_=model,labels_=positions)

    ## Regresión -- Random Forrest Regression with GridSearchCV--
    ## https://www.kaggle.com/code/sociopath00/random-forest-using-gridsearchcv/notebook
    if typereg == 'AutoRF':

        nestlist = []
        for i in range(0, 300, 50):
            nestlist.append(i)
        param_grid = { 
        'n_estimators': nestlist,
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth' : [4,5,6,7,8],
        'criterion' :['gini', 'entropy']}

        model = RandomForestRegressor(random_state=42)
        clf   = GridSearchCV(estimator=model, param_grid=param_grid, cv=5).fit(X, Y)
        model         = clf.best_estimator_
        results       = model.fit(X, Y)
        prediction_Y2 = results.predict(X_2)
        print_importances(model_=model,labels_=positions)

        print("n_estimators:       {}".format(model.n_estimators))
        print("max_features:       {}".format(model.max_features))
        print("max_depth:       {}".format(model.max_depth))
        print("criterion:       {}".format(model.criterion))
            
    print_serie2(neighbors2,prediction_Y2, 'Pronóstico - ' + typedist+' - ' + typereg ,'Demanda','Tiempo',7,4,'fig_2')

    t_reg = time.time() - t_sel - t_o

    return prediction_Y2, t_sel, t_reg

In [ ]:
## Parámetros de prueba.
vsel = 2016; vreg = 2016; k = 6 ; tol = 0.8
dias = 2
false_end = dias * vreg ## Usa dias*v como número de dias hacia atrás para hacer pruebas de dos ventanas hacia atras

## Dibujamos la ventana de histórico y los datos de prueba  
series = []; vacia = []
series.append(serie[len(serie)-12*false_end : len(serie)-false_end + vreg])
series.append(serie[len(serie)-12*false_end : len(serie)-false_end ])
print_serie2(serie_=series,prototipo_=vacia , title_='Serie de demanda eléctrica', ytitle_='Demanda (estandarizada)',xtitle_='Tiempo', sizex_=30, sizey_=7, namefile_='fig_1')

---

## Probamos el modelo con 'pearson' y 'RF'.



In [ ]:
if 1==1:
    x_train = serie[0 : len(serie) - false_end]
    y_test  = serie[len(serie) - false_end : len(serie) - false_end + vreg]
    pred_pea_rf, t_sel_pea_rf, t_reg_pea_rf = analogo_knn(x_train,vsel=vsel,vreg=vreg,k=k,tol=tol,typedist='pearson',typereg='RF')

    # Exactitud del modelo
    mae_pea_rf  = mean_absolute_error(y_test,pred_pea_rf)
    mape_pea_rf = mean_absolute_percentage_error(y_test,pred_pea_rf)

---

## Probamos el modelo con 'pearson' y 'AutoRF'.



In [ ]:
if 1==1:
    x_train = serie[0 : len(serie) - false_end]
    y_test  = serie[len(serie) - false_end : len(serie) - false_end + vreg]
    pred_pea_arf, t_sel_pea_arf, t_reg_pea_arf = analogo_knn(x_train,vsel=vsel,vreg=vreg,k=k,tol=tol,typedist='pearson',typereg='AutoRF')

    # Exactitud del modelo
    mae_pea_arf  = mean_absolute_error(y_test,pred_pea_arf)
    mape_pea_arf = mean_absolute_percentage_error(y_test,pred_pea_arf)

---
## Probamos el modelo con 'pearson' y 'OLS'.

In [ ]:
if 1==1:
    x_train = serie[0 : len(serie) - false_end]
    y_test  = serie[len(serie) - false_end : len(serie) - false_end + vreg]
    pred_pea_ols, t_sel_pea_ols, t_reg_pea_ols = analogo_knn(x_train,vsel=vsel,vreg=vreg,k=k,tol=tol,typedist='pearson',typereg='OLS')

    # Exactitud del modelo
    mae_pea_ols  = mean_absolute_error(y_test,pred_pea_ols)    
    mape_pea_ols = mean_absolute_percentage_error(y_test,pred_pea_ols)

---

## Probamos el modelo con 'pearson' y 'Voting'.

In [ ]:
if 1==0:
    x_train = serie[0 : len(serie) - false_end]
    y_test  = serie[len(serie) - false_end : len(serie) - false_end + vreg]
    pred_pea_vot, t_sel_pea_vot, t_reg_pea_vot = analogo_knn(x_train,vsel=vsel,vreg=vreg,k=k,tol=tol,typedist='pearson',typereg='Voting')

    # Exactitud del modelo
    mae_pea_vot = mean_absolute_error(y_test,pred_pea_vot)
    mape_pea_vot = mean_absolute_percentage_error(y_test,pred_pea_vot)



---



## Probamos el modelo con 'euclidian' y 'OLS' 

In [ ]:
if 1==0:
    x_train = serie[0 : len(serie) - false_end]
    y_test  = serie[len(serie) - false_end : len(serie) - false_end + vreg]
    pred_euc_ols, t_sel_euc_ols, t_reg_euc_ols = analogo_knn(x_train,vsel=vsel,vreg=vreg,k=k,tol=tol,typedist='euclidian',typereg='OLS')

    # Exactitud del modelo
    mae_euc_ols  = mean_absolute_error(y_test,pred_euc_ols)
    mape_euc_ols = mean_absolute_percentage_error(y_test,pred_euc_ols)

---

## Probamos el modelo con 'euclidian' y 'RF'.



In [ ]:
if 1==0:
    x_train = serie[0 : len(serie) - false_end]
    y_test  = serie[len(serie) - false_end : len(serie) - false_end + vreg]
    pred_euc_rf, t_sel_euc_rf, t_reg_euc_rf = analogo_knn(x_train,vsel=vsel,vreg=vreg,k=k,tol=tol,typedist='euclidian',typereg='RF')

    # Exactitud del modelo
    mae_euc_rf  = mean_absolute_error(y_test,pred_euc_rf)
    mape_euc_rf = mean_absolute_percentage_error(y_test,pred_euc_rf)

---

## Probamos el modelo con 'euclidian' y 'Voting'.

In [ ]:
if 1==0:
    x_train = serie[0 : len(serie) - false_end]
    y_test  = serie[len(serie) - false_end : len(serie) - false_end + vreg]
    pred_euc_vot, t_sel_euc_vot, t_reg_euc_vot = analogo_knn(x_train,vsel=vsel,vreg=vreg,k=k,tol=tol,typedist='euclidian',typereg='Voting')

    # Exactitud del modelo
    mae_euc_vot  = mean_absolute_error(y_test,pred_euc_vot)
    mape_euc_vot = mean_absolute_percentage_error(y_test,pred_euc_vot)

---

## Probamos el modelo con 'dtw' y 'OLS'.

In [ ]:
if 1==0:
    x_train = serie[0 : len(serie) - false_end]
    y_test  = serie[len(serie) - false_end : len(serie) - false_end + vreg]
    pred_dtw_ols, t_sel_dtw_ols, t_reg_dtw_ols = analogo_knn(x_train,vsel=vsel,vreg=vreg,k=k,tol=tol,typedist='dtw',typereg='OLS')

    # Exactitud del modelo
    mae_dtw_ols  = mean_absolute_error(y_test,pred_dtw_ols)
    mape_dtw_ols = mean_absolute_percentage_error(y_test,pred_dtw_ols)

---

## Probamos el modelo con 'dtw' y 'RF'.

In [ ]:
if 1==0:
    x_train = serie[0 : len(serie) - false_end]
    y_test  = serie[len(serie) - false_end : len(serie) - false_end + vreg]
    pred_dtw_rf, t_sel_dtw_rf, t_reg_dtw_rf = analogo_knn(x_train,vsel=vsel,vreg=vreg,k=k,tol=tol,typedist='dtw',typereg='RF')

    # Exactitud del modelo
    mae_dtw_rf  = mean_absolute_error(y_test,pred_dtw_rf)
    mape_dtw_rf = mean_absolute_percentage_error(y_test,pred_dtw_rf)

---

## Comparamos graficamente los resultados



In [ ]:
print('| SELECCIÓN      | REGRESIÓN      | MAE             | MAPE             | TIEMPO SELECCIÓN     | TIEMPO REGRESIÓN   |')
print('| :------------- | :------------- | -------------:  | -------------:   |-------------:        |-------------:        |')
print('| PEARSON        | OLS + STEP     |',str(trunc(mae_pea_ols,4)),'         |',str(trunc(mape_pea_ols,4)),'         |',str(trunc(t_sel_pea_ols,4)),'              |   ',str(trunc(t_reg_pea_ols,4)),     '|')
#print('| EUCLIDIAN      | OLS + STEP     |',str(trunc(mae_euc_ols,4)),'          |',str(trunc(mape_euc_ols,4)),'           |',str(trunc(t_sel_euc_ols,4)),'             |   ',str(trunc(t_reg_euc_ols,4)),    '|')
#print('| DTW            | OLS + STEP     |',str(trunc(mae_dtw_ols,4)),'         |',str(trunc(mape_dtw_ols,4)),'         |',str(trunc(t_sel_dtw_ols,4)),'            |   ',str(trunc(t_reg_dtw_ols,4)),    '|')
print('| PEARSON        | RF             |',str(trunc(mae_pea_rf,4)),'         |',str(trunc(mape_pea_rf,4)),'          |',str(trunc(t_sel_pea_rf,4)),'              |   ',str(trunc(t_reg_pea_rf,4)),    '|')
print('| PEARSON        | AutoRF            |',str(trunc(mae_pea_arf,4)),'         |',str(trunc(mape_pea_arf,4)),'          |',str(trunc(t_sel_pea_arf,4)),'              |   ',str(trunc(t_reg_pea_arf,4)),    '|')
#print('| EUCLIDIAN      | RF             |',str(trunc(mae_euc_rf,4)),'          |',str(trunc(mape_euc_rf,4)),'         |',str(trunc(t_sel_euc_rf,4)),'              |   ',str(trunc(t_reg_euc_rf,4)),     ' |')
#print('| DTW            | RF             |',str(trunc(mae_dtw_rf,4)),'         |',str(trunc(mape_dtw_rf,4)),'          |',str(trunc(t_sel_dtw_rf,4)),'            |   ',str(trunc(t_reg_dtw_rf,4)),    '|')
#print('| PEARSON        | Voting         |',str(trunc(mae_pea_vot,4)),'         |',str(trunc(mape_pea_vot,4)),'         |',str(trunc(t_sel_pea_vot,4)),'            |   ',str(trunc(t_reg_pea_vot,4)),    '|')
#print('| EUCLIDIAN        | Voting         |',str(trunc(mae_euc_vot,4)),'         |',str(trunc(mape_euc_vot,4)),'         |',str(trunc(t_sel_euc_vot,4)),'            |   ',str(trunc(t_reg_euc_vot,4)),    '|')

In [ ]:
fig  = plt.figure(figsize=(35,9))
axes = fig.add_subplot(1, 1, 1)
axes.legend()
CYAN = '#76ced6' ; LILA = '#777bd4'; VERDE='#17cb49'; LETRASNARA ='#ff8000'; NARA ='#ff8000'; AZUL='#168fff'; GRIS = '#808080'
mytitle = plt.title('Pronóstico de demanda eléctrica') # get the title property handler #plt.getp(title_obj)
plt.setp(mytitle,    color=LETRASNARA) #set the color of title to red
axes.xaxis.label.set_color(LETRASNARA)
axes.yaxis.label.set_color(LETRASNARA)
axes.tick_params(colors=LETRASNARA, which='both')

AUX = np.arange( len(y_test) )
plt.scatter ( AUX, y_test, s=25,marker='o', color = 'red', label = 'Y', alpha=1/2)
#axes.plot   ( y_test ,      '.-', color = 'red' , label = 'Y',         alpha=1/4)
#axes.plot   ( pred_euc_ols, '.-', color = CYAN  , label = 'Y_euc_ols', alpha=1/2)
axes.plot   ( pred_pea_ols, '.-', color = VERDE , label = 'Y_pea_ols', alpha=1/2)
#axes.plot   ( pred_dtw_ols, '.-', color = LILA  , label = 'Y_dtw_ols', alpha=1/2)
#axes.plot   ( pred_euc_rf, '.-',  color = NARA  , marker='x', label = 'Y_euc_rf', alpha=1/2)
axes.plot   ( pred_pea_rf,  '.-', color = GRIS  , marker='x', label = 'Y_pea_rf' , alpha=1/2)
axes.plot   ( pred_pea_arf,  '.-', color = AZUL  , marker='x', label = 'Y_pea_arf' , alpha=1/2)
#axes.plot   ( pred_dtw_rf, '.-',  color = AZUL  , label = 'Y_dtw_rf', alpha=1/2)
#axes.plot   ( pred_pea_vot, '.-', color = LILA  , label = 'Y_pea_vot', alpha=1/2)
#axes.plot   ( pred_euc_vot, '.-', color = VERDE  , label = 'Y_euc_vot', alpha=1/2)
#axes.plot  ( xspline[3], yspline[3] , '.-'  , color = VERDE   , label = 'orden=4'  ,alpha=1/2)
#plt.scatter( dfx_miss, dfy_miss, marker='x',  s=170 ,  color = 'red' , label = 'perdidos' ,alpha=1)

#plt.axis([0,  600, 7650, 9800])
#plt.axis([100,  200, 7650, 9500])
#plt.axis([400,  500, 7450, 9500])
#plt.axis([-2,  100, 8100, 9600])
#plt.axis([300, 400, 7900, 9500])

axes.spines['bottom'].set_color(LETRASNARA)
axes.spines['top'   ].set_color(LETRASNARA) 
axes.spines['right' ].set_color(LETRASNARA)
axes.spines['left'  ].set_color(LETRASNARA)
plt.legend()  
plt.xticks(rotation=90)
plt.savefig('fig_t15_ajuste_prono.png', transparent=True)
plt.show()

### Ligas interesantes
[link texHow to Develop a Random Forest Ensemble in Python](https://machinelearningmastery.com/random-forest-ensemble-in-python/)

[Random Forest using GridSearchCV](https://www.kaggle.com/code/sociopath00/random-forest-using-gridsearchcv/notebook)